In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Setup Chrome options
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0")

# Path to your ChromeDriver
service = Service("C:\\WebDriver\\bin\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

try:
    url = "https://www.dell.com/en-us/shop/dell-laptops/inspiron-15-laptop/spd/inspiron-15-3535-laptop/#ratings_section"
    driver.get(url)

    # Wait for the reviews element to be present
    reviews_locator = (By.CSS_SELECTOR, "dell-ratings-reviews")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located(reviews_locator))

    # Scroll the reviews element into view
    reviews_element = driver.find_element(*reviews_locator)
    driver.execute_script("arguments[0].scrollIntoView();", reviews_element)
    time.sleep(2)  # Allow time for any lazy-loaded content to appear

    # Access the shadow DOM and extract reviews
    script = """
        const shadowRoot = arguments[0].shadowRoot;
        const reviews = shadowRoot.querySelectorAll(".bv-content-summary-body-text");
        return Array.from(reviews).map(el => el.textContent.trim());
    """
    review_texts = driver.execute_script(script, reviews_element)

    # Output the reviews
    print("\nExtracted Reviews:\n")
    for i, review in enumerate(review_texts, 1):
        print(f"{i}. {review}\n")

except Exception as e:
    print("Error during scraping:", e)
finally:
    driver.quit()





Error during scraping: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7DDAA0315+28725]
	(No symbol) [0x00007FF7DDA02B00]
	(No symbol) [0x00007FF7DD89510A]
	(No symbol) [0x00007FF7DD8E93D2]
	(No symbol) [0x00007FF7DD8E95FC]
	(No symbol) [0x00007FF7DD933407]
	(No symbol) [0x00007FF7DD90FFEF]
	(No symbol) [0x00007FF7DD930181]
	(No symbol) [0x00007FF7DD90FD53]
	(No symbol) [0x00007FF7DD8DA0E3]
	(No symbol) [0x00007FF7DD8DB471]
	GetHandleVerifier [0x00007FF7DDDCF31D+3366973]
	GetHandleVerifier [0x00007FF7DDDE1300+3440672]
	GetHandleVerifier [0x00007FF7DDDD790D+3401261]
	GetHandleVerifier [0x00007FF7DDB6AACB+858091]
	(No symbol) [0x00007FF7DDA0E76F]
	(No symbol) [0x00007FF7DDA0A324]
	(No symbol) [0x00007FF7DDA0A4BD]
	(No symbol) [0x00007FF7DD9F8B89]
	BaseThreadInitThunk [0x00007FFD41BEE8D7+23]
	RtlUserThreadStart [0x00007FFD4375BF6C+44]



In [10]:
from crawl4ai import Crawl4AiClient, Job

# Initialize the Crawl4Ai client
client = Crawl4AiClient()

# Define a job that starts at the Newegg reviews page.
# (Replace the product ID with one for which reviews exist.)
job = Job(
    start_url="https://www.newegg.com/p/N82E16824028005/reviews",
    extraction_rules={
        # We'll extract reviews using a custom JS snippet.
        "reviews": {
            "js": """
                // Access the shadow DOM of the custom element
                const element = document.querySelector("dell-ratings-reviews");
                if (!element) return [];
                const shadow = element.shadowRoot;
                if (!shadow) return [];
                // Query for review text elements
                const reviewElements = shadow.querySelectorAll(".bv-content-summary-body-text");
                return Array.from(reviewElements).map(el => el.textContent.trim());
            """
        }
    }
)

# Run the job
result = client.run(job)

# Print out the extracted reviews
reviews = result.get("reviews", [])
print("Extracted Reviews from Newegg:")
for i, review in enumerate(reviews, 1):
    print(f"{i}. {review}")


ModuleNotFoundError: No module named 'pydantic_core._pydantic_core'